### 2. 증인/용의자 대화 프롬프트 제작
- 증인: 진실 정보(플레이어에게 힌트가 되는 정보)
    - 질문: 3~5회 제한
- 용의자: 왜곡된 정보(일부 진실을 변형하거나 은폐)
    - 질문: 제한 없음 or 제약 있음

In [ ]:
import ollama

In [ ]:
# 증인과의 대화
def case_file():
    chat_history = []
    system_instruction = """
    당신은 아래 범죄 사건의 증인입니다.
    플레이어의 질문에 해당 인물로서만 답해야 합니다.
    사실만을 진술해야 하며 플레이어에게 힌트가 되는 정보를 제공해야 합니다.

    사건 정보: {case} 
    
    ### 지시사항 ###
    피해자 가족, 이웃 등
    
    ### 출력 형식 ###

    """
    while True:
        user_input = input('프로파일러: ')
        if user_input == '종료':
            print("수사를 종료합니다.")
            break  

        chat_history.append({"role":"user", "content":user_input})
        response = ollama.chat(model='EEVE-Korean-10.8B', messages=chat_history)

        answer = response['message']['content']
        print('증인: ', answer)
        chat_history.append({"role":"assistant", "content":answer})

In [ ]:
# 용의자와의 대화
def case_file():
    chat_history = []
    f = open('case_file.json', 'r', encoding='utf-8')

    system_instruction = """
    당신은 아래 범죄 사건의 용의자입니다.
    플레이어의 질문에 해당 인물로서만 답해야 합니다.
    진술은 일부 진실을 왜곡하여 의도적으로 답할 수 있습니다.

    사건 정보: {case} 
    
    ### 지시사항 ###
    
    ### 출력 형식 ###

    """
    while True:
        user_input = input('프로파일러: ')
        if user_input == '종료':
            print("수사를 종료합니다.")
            break  

        chat_history.append({"role":"user", "content":user_input})
        response = ollama.chat(model='EEVE-Korean-10.8B', messages=chat_history)

        answer = response['message']['content']
        print('용의자: ', answer)
        chat_history.append({"role":"assistant", "content":answer})

In [ ]:
if __name__ == '__main__':
    f = open('case_file.json', 'r', encoding='utf-8')
    case = f.read()